<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8J%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본			   G_inc_so_model_노이2.pt
 산업분류자동화		   G_inc_so_model_노이.pt
'Colab Notebooks'	  'My Drive'
 G_inc_again_so_model.pt   Screenshot_20220401-001411_Chrome.jpg


In [3]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/J_spell_check.csv', sep = ",", encoding = "UTF-8")

## 데이터 전처리

In [4]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000065,J,58,582,사무실에서 고객 의뢰로 시스템소프트웨어 개발 및 공급
1,id_0000107,J,58,582,사업장에서 고객 의뢰를 받아 무인주차관리 시스템 개발 공급
2,id_0000157,J,59,592,인터넷으로 디지털 음원 음반 제작 유통
3,id_0000276,J,58,582,사무실에서 의뢰받아 소프트웨어 개발
4,id_0000338,J,58,582,사무실에서 고객 의뢰로 시스템소프트웨어 개발 및 공급
...,...,...,...,...,...
10857,id_0999684,J,58,582,사무실에서 고객 의뢰로 시스템소프트웨어 개발
10858,id_0999872,J,62,620,사무실에서 의뢰받아 소프트웨어 개발
10859,id_0999914,J,59,592,녹음시설 운영업
10860,id_0999959,J,58,582,사무실에서 고객 의뢰로 시스템 소프트웨어 개발


In [5]:
df['digit_3'].value_counts().sort_index()

581    1548
582    3771
591    1250
592     230
601      20
602     186
611     796
612     317
620    1919
631     456
639     369
Name: digit_3, dtype: int64

## **# 데이터 증분**

In [6]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [7]:
df0=df.loc[(df['digit_3'] == 581)]  #*2
df1=df.loc[(df['digit_3'] == 582)] 
df2=df.loc[(df['digit_3'] == 591)]  #*3
df3=df.loc[(df['digit_3'] == 592)]   #*15
df4=df.loc[(df['digit_3'] == 601)]   #*150
df5=df.loc[(df['digit_3'] == 602)]    #*15
df6=df.loc[(df['digit_3'] == 611)]    #*3
df7=df.loc[(df['digit_3'] == 612)]    #*8
df8=df.loc[(df['digit_3'] == 620)]   #*2
df9=df.loc[(df['digit_3'] == 631)]   #*8
df10=df.loc[(df['digit_3'] == 639)]  #*8

In [8]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)
df6=df6.reset_index(drop=True)
df7=df7.reset_index(drop=True)
df8=df8.reset_index(drop=True)
df9=df9.reset_index(drop=True)
df10=df10.reset_index(drop=True)

In [9]:
df_2=pd.concat([df0, df8]) # *2
df_3=pd.concat([df2, df6]) # *3
df_8=pd.concat([df7, df9, df10]) # *8 묶음
df_15=pd.concat([df3, df5]) # *15

In [10]:
df_2=df_2.reset_index(drop=True)
df_3=df_3.reset_index(drop=True)
df_8=df_8.reset_index(drop=True)
df_15=df_15.reset_index(drop=True)

In [11]:
df1.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000065,J,58,582,사무실에서 고객 의뢰로 시스템소프트웨어 개발 및 공급
1,id_0000107,J,58,582,사업장에서 고객 의뢰를 받아 무인주차관리 시스템 개발 공급
2,id_0000276,J,58,582,사무실에서 의뢰받아 소프트웨어 개발
3,id_0000338,J,58,582,사무실에서 고객 의뢰로 시스템소프트웨어 개발 및 공급
4,id_0000508,J,58,582,사무실에서 의뢰받아 소프트웨어 개발


In [13]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_2['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=1)
  id_list=[str(df_2['AI_id'][n]), 'p'+str(df_2['AI_id'][n])]
  digit_1_list=[df_2['digit_1'][n], df_2['digit_1'][n]]
  digit_2_list=[df_2['digit_2'][n], df_2['digit_2'][n]]
  digit_3_list=[df_2['digit_3'][n], df_2['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_2_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_2_to_insert, ignore_index=True)
  n=n+1

df_2=temp

In [14]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_3['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=[str(df_3['AI_id'][n]), 'p'+str(df_3['AI_id'][n]), 'p'+str(df_3['AI_id'][n])]
  digit_1_list=[df_3['digit_1'][n], df_3['digit_1'][n], df_3['digit_1'][n]]
  digit_2_list=[df_3['digit_2'][n], df_3['digit_2'][n], df_3['digit_2'][n]]
  digit_3_list=[df_3['digit_3'][n], df_3['digit_3'][n], df_3['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_3_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_3_to_insert, ignore_index=True)
  n=n+1

df_3=temp

In [15]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_8['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=7)
  id_list=[str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n]), 'p'+str(df_8['AI_id'][n])]
  digit_1_list=[df_8['digit_1'][n], df_8['digit_1'][n], df_8['digit_1'][n], df_8['digit_1'][n], df_8['digit_1'][n], df_8['digit_1'][n], df_8['digit_1'][n], df_8['digit_1'][n]]
  digit_2_list=[df_8['digit_2'][n], df_8['digit_2'][n], df_8['digit_2'][n], df_8['digit_2'][n], df_8['digit_2'][n], df_8['digit_2'][n], df_8['digit_2'][n], df_8['digit_2'][n]]
  digit_3_list=[df_8['digit_3'][n], df_8['digit_3'][n], df_8['digit_3'][n], df_8['digit_3'][n], df_8['digit_3'][n], df_8['digit_3'][n], df_8['digit_3'][n], df_8['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_8_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_8_to_insert, ignore_index=True)
  n=n+1

df_8=temp

In [16]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_15['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=14)
  id_list=[str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n]), 'p'+str(df_15['AI_id'][n])]
  digit_1_list=[df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n], df_15['digit_1'][n]]
  digit_2_list=[df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n], df_15['digit_2'][n]]
  digit_3_list=[df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n], df_15['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_15_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_15_to_insert, ignore_index=True)
  n=n+1

df_15=temp

In [17]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df4['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=149)
  id_list=[str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]),  'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n]), 'p'+str(df4['AI_id'][n])]
  digit_1_list=[df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n], df4['digit_1'][n]]
  digit_2_list=[df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n], df4['digit_2'][n]]
  digit_3_list=[df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n], df4['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df4_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df4_to_insert, ignore_index=True)
  n=n+1

df4=temp

기존 데이터 합치기

In [18]:
df=pd.concat([df1, df_2, df_3, df_8, df_15, df4])

In [19]:
df['digit_3'].value_counts().sort_index()

581.0    3096
582.0    3771
591.0    3750
592.0    3450
601.0    3000
602.0    2790
611.0    2388
612.0    2536
620.0    3838
631.0    3648
639.0    2952
Name: digit_3, dtype: int64

In [ ]:
df

In [20]:
df.to_csv('df_J_inc_again.csv', index=False, encoding='utf-8-sig')